In [100]:
# Load libraries
import numpy as np
import pandas as pd

rawdf = pd.read_csv('data/raw_data_macrozoobenthos_1980-2005.csv', header=0)
# rawdf.columns

# Reformat raw data into site by abundance matrix

In [ ]:
# Extract relevant columns to create Site by Abundance matrix
data = rawdf[['scientificnameaccepted', \
    'decimallatitude', 'decimallongitude', \
    'yearcollected',  'monthcollected', 'daycollected', \
     'minimumdepthinmeters', 'maximumdepthinmeters', 'Count (Dmnless)'
    ]]

# Create variable Site using latitude and longitude
data['site'] = 'site_' + data['decimallatitude'].astype(str) + '_' + data['decimallongitude'].astype(str)
data = data[['site', 'scientificnameaccepted', \
    'decimallatitude', 'decimallongitude', \
    'yearcollected',  'monthcollected', 'daycollected', \
     'minimumdepthinmeters', 'maximumdepthinmeters', 'Count (Dmnless)'
    ]]

# Transform data to Site by Abundance matrix
## --- agregated over date, month, year
# data = data.groupby(['site', 'scientificnameaccepted'])['Count (Dmnless)'].sum().reset_index()

# # Transform data to Site by Abundance matrix
# data = data.pivot(index='site', columns='scientificnameaccepted', values='Count (Dmnless)')
# data.fillna(0, inplace=True

# data.to_csv('data/sps_macrozoobenthos_allyear.csv')

## ----- Now we will create a matrix over time
# data = data.groupby(['site', 'scientificnameaccepted', 'yearcollected', 'monthcollected', 'daycollected'])['Count (Dmnless)'].sum().reset_index()
# data["MMDDYYY"] = data["monthcollected"].astype(str) + '-' + data["daycollected"].astype(str) + '-' + data["yearcollected"].astype(str)
# # data[['site', 'MMDDYYY', 'scientificnameaccepted', 'Count (Dmnless)']].to_csv('data/sps_macrozoobenthos_timeseries.csv')

# # Create column quarter of the year
# data['quarter'] = pd.to_datetime(data['MMDDYYY']).dt.quarter
# # Aggregate over quarter
# data_quarter = data.groupby(['site', 'scientificnameaccepted', 'quarter', "yearcollected"])['Count (Dmnless)'].sum().reset_index()
# data_quarter.to_csv('data/sps_macrozoobenthos_timeseries_quarter.csv')

# # Aggregate over year
# data_year = data.groupby(['site', 'scientificnameaccepted', "yearcollected"])['Count (Dmnless)'].sum().reset_index()
# data_year.to_csv('data/sps_macrozoobenthos_timeseries_year.csv')

C:\Users\ttran\AppData\Local\Temp\ipykernel_26224\482386081.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['site'] = 'site_' + data['decimallatitude'].astype(str) + '_' + data['decimallongitude'].astype(str)


# Load environemental data SUV

With the script below, I determined that the datasets variables are
- SUR contains 'VARIABLES', 'Depth', 'Temperatur', 'Salinity', 'xCO2atm', 'Latitude', 'Longitude', 'JulianDay'
- UOR contains 'VARIABLES', 'Depth', 'Temperatur', 'Salinity', 'Chlorophyl', 'Latitude', 'Longitude', 'JulianDay'
- PFL, XBT does not contain extra necessary data more than : 'VARIABLES', 'Depth', 'Temperatur', 'Salinity'
- CTD contains 'VARIABLES', 'Depth', 'Temperatur', 'Salinity', 'Oxygen', 'Chlorophyl'
- OSD contains 'VARIABLES', 'Depth', 'Temperatur', 'Salinity', 'Oxygen', 'Phosphate', 'Silicate', 'Nitrate', 'Alkalinity', 'tCO2', 'Helium', 'Neon', 'CFC11', 'CFC12

In [88]:
with open('data/ocldb1738015069.1150601.OSD.csv', mode="r") as file:
    filedata = file.readlines()
# max_L, L = 0, []
# for l in filedata:
#     if "VARIABLES" in l:
#         # print(list(map(lambda x: x.strip(), l.split(","))))
#         if max_L <= len(l.split(",")):
#             max_L = len(l.split(","))
#             L = []
#             for i in list(map(lambda x: x.strip(), l.split(","))):
#                 if len(i) >1:
#                     L.append(i)
                    
# Check how the variables are ordered
# for l in filedata:
#     if "VARIABLES" in l:
#         print(list(map(lambda x: x.strip(), l.split(","))))
    

## Codes to extract data from raw datasets

In [99]:
import pandas as pd
import re

# Load the file
# file_path = "data/ocldb1738015069.1150601.OSD.csv"
# file_path = "data/ocldb1738015069.1150601.CTD.csv"
file_path = "data/ocldb1738015069.1150601.SUR.csv"

# Function to preprocess the data
def prep_data(file_path, save=True):
    # Read the file as a list of lines
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    # Initialize variables
    data = []
    current_record = {}
    depth_data = []
    variable_names = []

    # Pattern matching for the header fields
    patterns = {
        "CAST": r"CAST\s+,,\s*(\d+)",
        "Year": r"Year\s+,,\s*(\d+)",
        "Month": r"Month\s+,,\s*(\d+)",
        "Day": r"Day\s+,,\s*(\d+)",
        "Longitude": r"Longitude\s+,,\s*([\d.]+)",
        "Latitude": r"Latitude\s+,,\s*([\d.]+)",
    }

    # Process each line (after VARIABLES section)
    for line in lines:
        # Extract metadata
        for key, pattern in patterns.items():
            match = re.search(pattern, line)
            if match:
                current_record[key] = float(match.group(1)) if "." in match.group(1) else int(match.group(1))

        # Detect the start of variables section
        if line.startswith("VARIABLES"):
            depth_data = []  # Reset depth data
            variable_names = [v.strip() for v in line.split(",")[1:] if v.strip()]

        # Extract depth and all available variable values dynamically
        values = line.split(",")
        if len(values) > 1 and values[0].strip().isdigit():  # Ensure numeric depth
            try:
                depth_entry = {}
                for i, var in enumerate(variable_names):
                    try:
                        value = float(values[i + 1].strip()) if "---" not in values[i + 1] else None
                        depth_entry[var] = value
                    except ValueError:
                        depth_entry[var] = None  # Handle non-numeric values
                depth_data.append(depth_entry)
            except ValueError:
                continue

        # End of a record, process and store it
        if line.startswith("END OF VARIABLES SECTION"):
            if depth_data:
                # Compute averages for all variables dynamically
                averages = {}
                for var in variable_names:
                    valid_values = [entry[var] for entry in depth_data if entry[var] is not None]
                    averages[f"Avg_{var}"] = sum(valid_values) / len(valid_values) if valid_values else None

                # Store processed record
                current_record.update(averages)
                data.append(current_record.copy())

    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    if save :
        file_path = file_path.replace(".csv", "_processed.csv")
        # Save the processed data
        df.to_csv(file_path, index=False)
    # return df

# Uncomment to run the function
# prep_data(file_path, save=True)